# Retrieving data for the performance of teams based on roles
This script does the following:
1. Creates a dataframe with `team_id` as the key
2. Joins the `matches_NA` and `games_NA` dataframes in an aggreagated dataframe `df_aggregate`
3. Queries the match win-rate of each team from `df_aggregate`
4. Queries the round win-rate of each team from `df_aggregate`
5. Queries the performance of each team from `df_aggregate` based on the following roles: 
    - Controller
    - Duelist
    - Sentinel
    - Initiator
6. Outputs a `pandas` dataframe with the relevant queries

### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain

### Load the Dataset

In [2]:
df_matches = pd.read_csv('matches_NA.csv')
df_games = pd.read_csv('games_NA.csv')

### Getting each unique team

In [3]:
teams_0 = df_matches.team0_ID.unique()
df_0 = pd.DataFrame(teams_0, columns = ['team_ID'])
teams_1 = df_matches.team1_ID.unique()
df_1 = pd.DataFrame(teams_1, columns = ['team_ID'])
df = pd.concat([df_0, df_1])
df = df.drop_duplicates()

### Get win-rate of each team per game(map)

In [4]:
team_wins = []
team_losses = []
for team in df.team_ID:
    wins = df_games.query('winner_ID == @team')
    team_wins.append(wins.shape[0])
    losses = df_games.query('loser_ID == @team')
    team_losses.append(losses.shape[0])
    
df.insert(loc=1, column="num_wins", value=team_wins)
df.insert(loc=2, column="num_losses", value=team_losses)

df["match_winrate"] = (df.num_wins / (df.num_wins + df.num_losses))

### Join the matches and games dataframes

In [5]:
df_aggregate = pd.merge(df_matches, df_games, on='match_id')

### Get controller performance of each team per game

In [6]:
controllers = ["astra", "brimstone", "omen", "viper"]

team_ACS = []
team_KAST = []
team_round_winrate = []
for team in df.team_ID:
    # Get instances where this team is team0
    games = df_aggregate.query('team0_ID == @team')
    
    controller_ACS = []
    controller_KAST = []

    rounds_won_as_t0 = sum((games.team0_score_y).values)
    rounds_lost_as_t0 = sum((games.team1_score_y).values)
    
    # Get performance for controllers
    for i in range(0, 5):
        current_query = 'player' + str(i) + '_agent in @controllers'
        if not ((p := games.query(current_query)).empty):
            controller_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            controller_KAST.append((p['player' + str(i) + '_KAST_y']).values)        
    
    # Get instances where this team is team1
    games = df_aggregate.query('team1_ID == @team')
    
    rounds_won_as_t1 = sum((games.team1_score_y).values)
    rounds_lost_as_t1 = sum((games.team0_score_y).values)
    
    # Get performance for controllers
    for i in range(5, 10):
        current_query = 'player' + str(i) + '_agent in @controllers'
        if not ((p := games.query(current_query)).empty):
            controller_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            controller_KAST.append((p['player' + str(i) + '_KAST_y']).values) 
        
    round_winrate = (rounds_won_as_t0 + rounds_won_as_t1)/(rounds_won_as_t0 + rounds_won_as_t1 + rounds_lost_as_t0 + rounds_lost_as_t1)
    team_round_winrate.append(round_winrate)

    flatten_list = list(chain.from_iterable(controller_ACS))
    flatten_list = [ int(x) for x in flatten_list ]
    team_ACS.append(np.average(flatten_list))
    
    flatten_list = list(chain.from_iterable(controller_KAST))
    flatten_list = [ int(x) for x in flatten_list ]
    team_KAST.append(np.average(flatten_list))

df.insert(loc=4, column="round_winrate", value=team_round_winrate)
df.insert(loc=5, column="controller_ACS", value = team_ACS)
df.insert(loc=6, column="controller_KAST", value = team_KAST)

### Get duelist performance of each team per game

In [7]:
duelists = ["reyna", "yoru", "jett", "phoenix", "raze", "neon"]

team_ACS = []
team_KAST = []
for team in df.team_ID:
    # Get instances where this team is team0
    games = df_aggregate.query('team0_ID == @team')
    
    duelist_ACS = []
    duelist_KAST = []
    
    # Get performance for duelists
    for i in range(0, 5):
        current_query = 'player' + str(i) + '_agent in @duelists'
        if not ((p := games.query(current_query)).empty):
            duelist_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            duelist_KAST.append((p['player' + str(i) + '_KAST_y']).values)  
        
    # Get instances where this team is team1
    games = df_aggregate.query('team1_ID == @team')
    
    # Get performance for duelists
    for i in range(5, 10):
        current_query = 'player' + str(i) + '_agent in @duelists'
        if not ((p := games.query(current_query)).empty):
            duelist_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            duelist_KAST.append((p['player' + str(i) + '_KAST_y']).values)  

    flatten_list = list(chain.from_iterable(duelist_ACS))
    flatten_list = [ int(x) for x in flatten_list ]
    team_ACS.append(np.average(flatten_list))
    
    flatten_list = list(chain.from_iterable(duelist_KAST))
    flatten_list = [ int(x) for x in flatten_list ]
    team_KAST.append(np.average(flatten_list))

df.insert(loc=7, column="duelist_ACS", value = team_ACS)
df.insert(loc=8, column="duelist_KAST", value = team_KAST)

### Get sentinel performance of each team per game

In [8]:
sentinels = ["sage", "cypher", "killjoy", "chamber"]

team_ACS = []
team_KAST = []
for team in df.team_ID:
    # Get instances where this team is team0
    games = df_aggregate.query('team0_ID == @team')
    
    sentinel_ACS = []
    sentinel_KAST = []
    
    # Get performance for sentinels
    for i in range(0, 5):
        current_query = 'player' + str(i) + '_agent in @sentinels'
        if not ((p := games.query(current_query)).empty):
            sentinel_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            sentinel_KAST.append((p['player' + str(i) + '_KAST_y']).values)  
        
    # Get instances where this team is team1
    games = df_aggregate.query('team1_ID == @team')
    
    # Get performance for sentinels
    for i in range(5, 10):
        current_query = 'player' + str(i) + '_agent in @sentinels'
        if not ((p := games.query(current_query)).empty):
            sentinel_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            sentinel_KAST.append((p['player' + str(i) + '_KAST_y']).values)  

    flatten_list = list(chain.from_iterable(sentinel_ACS))
    flatten_list = [ int(x) for x in flatten_list ]
    team_ACS.append(np.average(flatten_list))
    
    flatten_list = list(chain.from_iterable(sentinel_KAST))
    flatten_list = [ int(x) for x in flatten_list ]
    team_KAST.append(np.average(flatten_list))

df.insert(loc=9, column="sentinel_ACS", value = team_ACS)
df.insert(loc=10, column="sentinel_KAST", value = team_KAST)

### Get initiator performance of each team per game

In [9]:
initiators = ["breach", "sova", "skye", "kayo"]

team_ACS = []
team_KAST = []
for team in df.team_ID:
    # Get instances where this team is team0
    games = df_aggregate.query('team0_ID == @team')
    
    initiator_ACS = []
    initiator_KAST = []
    
    # Get performance for initiators
    for i in range(0, 5):
        current_query = 'player' + str(i) + '_agent in @initiators'
        if not ((p := games.query(current_query)).empty):
            initiator_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            initiator_KAST.append((p['player' + str(i) + '_KAST_y']).values)  
        
    # Get instances where this team is team1
    games = df_aggregate.query('team1_ID == @team')
    
    # Get performance for initiators
    for i in range(5, 10):
        current_query = 'player' + str(i) + '_agent in @initiators'
        if not ((p := games.query(current_query)).empty):
            initiator_ACS.append((p['player' + str(i) + '_ACS_y']).values)
            initiator_KAST.append((p['player' + str(i) + '_KAST_y']).values) 

    flatten_list = list(chain.from_iterable(initiator_ACS))
    flatten_list = [ int(x) for x in flatten_list ]
    team_ACS.append(np.average(flatten_list))
    
    flatten_list = list(chain.from_iterable(initiator_KAST))
    flatten_list = [ int(x) for x in flatten_list ]
    team_KAST.append(np.average(flatten_list))

df.insert(loc=11, column="initiator_ACS", value = team_ACS)
df.insert(loc=12, column="initiator_KAST", value = team_KAST)

In [10]:
df

,team_ID,num_wins,num_losses,match_winrate,round_winrate,controller_ACS,controller_KAST,duelist_ACS,duelist_KAST,sentinel_ACS,sentinel_KAST,initiator_ACS,initiator_KAST
0,/team/8127/optic-gaming,16,10,0.615385,0.532443,199.314286,75.171429,232.636364,73.227273,204.133333,68.933333,192.232558,69.883721
1,/team/6675/the-guard,31,12,0.720930,0.566975,205.250000,74.083333,236.232558,73.441860,201.543478,74.130435,210.484848,75.060606
2,/team/533/xset,26,10,0.722222,0.573050,181.344828,73.431034,249.456522,72.565217,214.625000,74.416667,197.980769,74.288462
3,/team/188/cloud9,17,11,0.607143,0.559380,188.935484,74.516129,262.821429,75.392857,211.187500,72.406250,200.938776,77.979592
4,/team/642/luminosity,30,22,0.576923,0.536540,200.267606,73.408451,230.348485,72.439394,198.000000,69.888889,203.717949,73.474359
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,/team/7816/5-of-a-kind,0,3,0.000000,0.235294,145.000000,50.000000,155.250000,46.000000,103.000000,35.000000,140.500000,39.500000
149,/team/7856/s4-ge,0,2,0.000000,0.161290,186.333333,55.000000,140.000000,49.500000,162.000000,42.000000,119.750000,43.750000
150,/team/7854/girl-kissers,0,2,0.000000,0.235294,121.500000,48.500000,227.000000,48.500000,148.000000,42.500000,123.500000,50.750000
151,/team/7818/nerd-emoji,0,2,0.000000,0.257143,158.500000,64.000000,175.000000,56.333333,134.000000,53.500000,130.333333,51.333333
